In [1]:
import pandas as pd
import os
import datetime as dt
import numpy as np
from pprint import pp

In [2]:
#use folder path as the path to folder with all the yearly storm data
folder_path='./Resources/'
dfs=[]
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Read CSV file into a dataframe
        df = pd.read_csv(file_path, low_memory=False)
        # Append dataframe to the list
        dfs.append(df)
combined_storm_df = pd.concat(dfs, ignore_index=True)

combined_storm_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201402,18,1000,201402,18,2000,83473.0,503953,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure developing south of Long Island a...,Eight to twelve inches of snow fell across eas...,CSV
1,201402,5,300,201402,5,2300,83491.0,504065,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure moving off the mid-Atlantic coast...,Six to twelve inches of snow fell across easte...,CSV
2,201401,18,1000,201401,19,700,82185.0,494521,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure brought a brief period of heavy s...,Four to eight inches of snow fell across easte...,CSV
3,201411,26,1000,201411,27,1000,91728.0,549746,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong coastal storm moved up the east coast...,Six to eight inches of snow fell across easter...,CSV
4,201402,13,630,201402,14,800,83476.0,503982,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A significant winter storm brought six to twel...,Five to eight inches of snow fell across easte...,CSV


In [3]:
len(combined_storm_df)

1671202

In [4]:
# filter to just texas
us_state = "TEXAS"

combined_storm_df = combined_storm_df.loc[combined_storm_df['STATE'] == us_state, :]

print(len(combined_storm_df), "storm records in", us_state)
combined_storm_df.head()

125244 storm records in TEXAS


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
307,201409,20,1500,201409,20,1540,89769.0,540290,TEXAS,48.0,...,1.0,SW,MENTONE,31.6898,-103.6120,31.6930,-103.6099,Remains of Tropical Storm Odile were over the ...,Heavy rain fell across Loving County and produ...,CSV
308,201409,20,2058,201409,20,2140,89769.0,540394,TEXAS,48.0,...,15.0,SE,ANDREWS CO ARPT,32.1669,-102.3464,32.1724,-102.3499,Remains of Tropical Storm Odile were over the ...,Heavy rain fell across Andrews County and prod...,CSV
309,201409,21,722,201409,21,830,89819.0,540409,TEXAS,48.0,...,1.0,N,MIDLAND,32.0181,-102.0819,32.0204,-102.0825,The remnants of Tropical Storm Odile and an as...,Heavy rain fell across Midland County and prod...,CSV
325,201409,23,645,201409,23,800,90016.0,541042,TEXAS,48.0,...,19.0,E,PINE SPGS,31.8677,-104.5267,31.8849,-104.4848,Weak upper ridging was to the west of the regi...,Heavy rain fell across Culberson County and pr...,CSV
326,201409,26,1200,201409,26,1800,90018.0,541049,TEXAS,48.0,...,10.0,NE,ORLA,31.8554,-103.9252,31.9300,-103.7769,An upper level low pressure system that moved ...,Heavy rains from the previous week of rainfall...,CSV


In [5]:
#remove the columns we obviously do not need
combined_storm_df = combined_storm_df[['BEGIN_DATE_TIME','DAMAGE_PROPERTY','EVENT_TYPE']]
combined_storm_df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_TYPE
307,20-SEP-14 15:00:00,0.50K,Flash Flood
308,20-SEP-14 20:58:00,0.30K,Flash Flood
309,21-SEP-14 07:22:00,1.00K,Flash Flood
325,23-SEP-14 06:45:00,0.20K,Flood
326,26-SEP-14 12:00:00,0.50K,Flood


In [6]:
#fill the null values in 'DAMAGE_PROPERTY' column with 0s
combined_storm_df['DAMAGE_PROPERTY'] = combined_storm_df['DAMAGE_PROPERTY'].fillna(0)


In [7]:
len(combined_storm_df)

125244

In [8]:
combined_storm_df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_TYPE
307,20-SEP-14 15:00:00,0.50K,Flash Flood
308,20-SEP-14 20:58:00,0.30K,Flash Flood
309,21-SEP-14 07:22:00,1.00K,Flash Flood
325,23-SEP-14 06:45:00,0.20K,Flood
326,26-SEP-14 12:00:00,0.50K,Flood


In [9]:
#function to convert the values in 'DAMAGE_PROPERTY' to float
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

In [10]:
combined_storm_df['DAMAGE_PROPERTY'] = combined_storm_df['DAMAGE_PROPERTY'].apply(value_to_float)

In [11]:
#function to convert values in 'BEGIN_DATE_TIME' AND 'END_DATE_TIME' to datetime variables
def convert_to_datetime(date_string):
    format_string = '%d-%b-%y %H:%M:%S'
    return dt.datetime.strptime(date_string, format_string)

In [12]:
combined_storm_df['BEGIN_DATE_TIME'] = combined_storm_df['BEGIN_DATE_TIME'].apply(convert_to_datetime)

## Aggregating Storm Data by Month

In [13]:
# filter out any dates before 2000
combined_storm_df = combined_storm_df.loc[combined_storm_df['BEGIN_DATE_TIME'] > dt.datetime(2000, 1, 1)]

len(combined_storm_df)

104554

In [14]:
# Adding month column from begin_datetime
combined_storm_df['BEGIN_MONTH'] = combined_storm_df['BEGIN_DATE_TIME'].dt.to_period('M')

combined_storm_df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_TYPE,BEGIN_MONTH
307,2014-09-20 15:00:00,500.0,Flash Flood,2014-09
308,2014-09-20 20:58:00,300.0,Flash Flood,2014-09
309,2014-09-21 07:22:00,1000.0,Flash Flood,2014-09
325,2014-09-23 06:45:00,200.0,Flood,2014-09
326,2014-09-26 12:00:00,500.0,Flood,2014-09


In [15]:
# verifying dates in range
print(combined_storm_df['BEGIN_MONTH'].max())
print(combined_storm_df['BEGIN_MONTH'].min())


2023-11
2000-01


In [16]:
# grabbing df info to inform aggregation functions
combined_storm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104554 entries, 307 to 1624811
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BEGIN_DATE_TIME  104554 non-null  datetime64[ns]
 1   DAMAGE_PROPERTY  104554 non-null  float64       
 2   EVENT_TYPE       104554 non-null  object        
 3   BEGIN_MONTH      104554 non-null  period[M]     
dtypes: datetime64[ns](1), float64(1), object(1), period[M](1)
memory usage: 4.0+ MB


In [17]:
# aggregating our storm data

# setting the aggregation functions via dict to pass in
aggs = {
    "DAMAGE_PROPERTY": 'sum',
    "EVENT_TYPE": "count",
}

agg_storm_df = combined_storm_df.groupby('BEGIN_MONTH').agg(aggs).rename(columns={"EVENT_TYPE": "EVENT_COUNT"})

agg_storm_df.head(24)

,DAMAGE_PROPERTY,EVENT_COUNT
BEGIN_MONTH,,
2000-01,0.000000e+00,76
2000-02,2.088900e+07,177
2000-03,1.454270e+07,731
2000-04,1.729150e+07,616
2000-05,5.461070e+07,635
2000-06,1.256400e+07,341
2000-07,1.749000e+06,226
2000-08,1.920000e+06,327
2000-09,2.694000e+06,322


## Pulling in/Merging Real Estate Data

In [18]:
# previewing our real estate data

# since the data is formatted a bit weirdly, I think this is the easiest way to process everything
import csv

re_csv_path = os.path.join("Resources_RealEstate", "State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")

with open(re_csv_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    # drop header
    for i in range(6):
        print(next(reader))
    
    print("csv month data length", len(next(reader)[5:]))



print("storm df length:", len(agg_storm_df))

['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07-3

In [19]:
# parsing the data for our real estate DF

# headers containing state name
state_hdrs = []
# real estate data
re_data = pd.DataFrame(columns=["STATE", "MONTH", "ZHVI"])

with open(re_csv_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    # drop header
    csv_headers = next(reader)
    for row in reader:
        state_hdrs.append(row[2])
        # iterate over rest of row
        for i in np.arange(5, len(row), 1):
            # print(i)
            re_data = pd.concat([re_data, pd.DataFrame({"STATE": [row[2]], "MONTH": [csv_headers[i][:7]], "ZHVI": [row[i]]})], ignore_index=True)

re_data.head()

,STATE,MONTH,ZHVI
0,California,2000-01,190252.3290963135
1,California,2000-02,190891.8239611473
2,California,2000-03,191755.29862276258
3,California,2000-04,193629.64465037093
4,California,2000-05,195808.28955393317


In [20]:
re_data.info()

len(re_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790 entries, 0 to 14789
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STATE   14790 non-null  object
 1   MONTH   14790 non-null  object
 2   ZHVI    14790 non-null  object
dtypes: object(3)
memory usage: 346.8+ KB


14790

In [21]:
print(re_data["MONTH"].min(), "to", re_data["MONTH"].max())

2000-01 to 2024-02


In [22]:
re_data = re_data.loc[re_data["MONTH"].astype('datetime64[s]') < dt.datetime(2024, 1, 1), :]

re_data.info()
re_data["MONTH"].max()

<class 'pandas.core.frame.DataFrame'>
Index: 14688 entries, 0 to 14787
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STATE   14688 non-null  object
 1   MONTH   14688 non-null  object
 2   ZHVI    14688 non-null  object
dtypes: object(3)
memory usage: 459.0+ KB


'2023-12'

In [23]:
re_data.loc[re_data["ZHVI"] == "", :].count()

STATE    228
MONTH    228
ZHVI     228
dtype: int64

In [24]:
re_data_tex = re_data.loc[re_data["STATE"] == "Texas", :].reset_index(drop=True)
re_data_tex.tail()

,STATE,MONTH,ZHVI
283,Texas,2023-08,299846.5848115864
284,Texas,2023-09,299884.498335656
285,Texas,2023-10,299654.3838471279
286,Texas,2023-11,299166.0208531806
287,Texas,2023-12,298562.7916413898


In [25]:
re_data_tex["MONTH"] = re_data_tex["MONTH"].astype("period[M]")
re_data_tex["ZHVI"] = re_data_tex["ZHVI"].astype("float")
re_data_tex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype    
---  ------  --------------  -----    
 0   STATE   288 non-null    object   
 1   MONTH   288 non-null    period[M]
 2   ZHVI    288 non-null    float64  
dtypes: float64(1), object(1), period[M](1)
memory usage: 6.9+ KB


In [26]:
re_data_tex = re_data_tex.set_index("MONTH")
re_data_tex.head()

,STATE,ZHVI
MONTH,,
2000-01,Texas,110404.230186
2000-02,Texas,110464.226696
2000-03,Texas,110493.325944
2000-04,Texas,110637.748090
2000-05,Texas,110731.278136


In [27]:
agg_storm_df = agg_storm_df.rename(columns={"BEGIN_MONTH": "MONTH"})


# merging dfs together
re_storm_data = pd.merge(re_data_tex, agg_storm_df, how="left", left_index=True, right_index=True).drop(columns=["STATE"])

re_storm_data.info()
re_storm_data.head()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 288 entries, 2000-01 to 2023-12
Freq: M
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ZHVI             288 non-null    float64
 1   DAMAGE_PROPERTY  287 non-null    float64
 2   EVENT_COUNT      287 non-null    float64
dtypes: float64(3)
memory usage: 17.1 KB


,ZHVI,DAMAGE_PROPERTY,EVENT_COUNT
MONTH,,,
2000-01,110404.230186,0.0,76.0
2000-02,110464.226696,20889000.0,177.0
2000-03,110493.325944,14542700.0,731.0
2000-04,110637.748090,17291500.0,616.0
2000-05,110731.278136,54610700.0,635.0


In [28]:
re_storm_data.index.max()

Period('2023-12', 'M')

In [29]:
# checking which months have 0 property damage recorded from storms
re_storm_data.loc[re_storm_data["DAMAGE_PROPERTY"] == 0, :]

,ZHVI,DAMAGE_PROPERTY,EVENT_COUNT
MONTH,,,
2000-01,110404.230186,0.0,76.0
2003-01,117550.763021,0.0,26.0
2017-11,191721.666367,0.0,31.0
2019-12,212792.772192,0.0,56.0


## Real Estate: full data + texas specific data with storms: Write to CSV
***

In [31]:
re_storm_data.to_csv(os.path.join("Resources_Output", "data_texas_property_zhvi_and_storm_damage_2000-2023.csv"))
re_data.to_csv(os.path.join("Resources_Output", "data_zhvi_all_states_2000-2023.csv"))

## Event Type Double Checking

In [ ]:
df["EVENT_TYPE"].value_counts()

In [ ]:
grouped = df.groupby(["EVENT_TYPE"])
grouped.sum(["DAMAGE_PROPERTY"]).drop(columns=["BEGIN_LAT", "BEGIN_LON", "END_LAT", "END_LON"]).sort_values("DAMAGE_PROPERTY", ascending=False)

In [ ]:
df["EVENT_TYPE"].unique()

In [ ]:
category_trans_dict = {
    'THUNDERSTORM WINDS/FLOODING': 'Flood',
    'HAIL/ICY ROADS': 'Hail',
    'HAIL FLOODING': 'Flood',
    'THUNDERSTORM WINDS/FLASH FLOOD': 'Flash Flood',
    'THUNDERSTORM WINDS LIGHTNING': 'Lightning',
    'THUNDERSTORM WIND/ TREES': 'Thunderstorm Wind',
    'THUNDERSTORM WIND/ TREE': 'Thunderstorm Wind',
    'THUNDERSTORM WINDS FUNNEL CLOU': 'Funnel Cloud',
    'TORNADO/WATERSPOUT': 'Waterspout',
    'THUNDERSTORM WINDS/HEAVY RAIN': 'Heavy Rain',
    'THUNDERSTORM WINDS HEAVY RAIN': 'Heavy Rain',
    'THUNDERSTORM WINDS/ FLOOD': 'Flood',
}


rest_of_cats = ['Tornado', 'Thunderstorm Wind', 'Hail', 'Winter Storm', 'Cold/Wind Chill',
       'Heavy Snow', 'Flood', 'High Wind', 'Flash Flood', 'Blizzard',
       'Ice Storm', 'Lightning', 'Frost/Freeze', 'Heavy Rain',
       'Strong Wind', 'Coastal Flood', 'Wildfire', 'Funnel Cloud',
       'Winter Weather', 'Waterspout', 'Drought', 'Debris Flow', 'Heat',
       'High Surf', 'Tropical Storm', 'Dust Devil', 'Dense Fog',
       'Hurricane (Typhoon)', 'Marine High Wind', 'Dust Storm',
       'Storm Surge/Tide', 'Lake-Effect Snow', 'Rip Current', 'Avalanche',
       'Seiche', 'Extreme Cold/Wind Chill', 'Excessive Heat', 'Tsunami',
       'Sleet', 'Freezing Fog', 'Lakeshore Flood',
       'Astronomical Low Tide', 'Tropical Depression', 'Dense Smoke',
       'Sneakerwave', 'Hurricane']

all_event_transforms = {item: item for item in rest_of_cats}
all_event_transforms.update(category_trans_dict)

pp(all_event_transforms.keys())

In [ ]:
df["EVENT_TYPE"] = df["EVENT_TYPE"].map(all_event_transforms)

df.head()

In [ ]:
# verify remaining categories
df["EVENT_TYPE"].unique()

### Some Thoughts on Event Type Consolidation
***

We may want to consider more merges including:

Hurricane (Typhoon) -> 'Hurricane'

All Floods (except maybe flash floods) -> Flood

(storm floods already filtered are a bit ambiguous in this case)

**Ideally, we either want to drop unneeded categories or run them through an unsupervised clustering algorithm to bring the total category count down to <10 for easier neural network processing**

# One-Hot Encoding, final preprocessing DF
***

In [ ]:
# double check the columns to see what we need encoded
df.head()

Columns to encode: "REGION", "EVENT_TYPE"

Could also be worthwhile to target data with only lat-long info and run clustering there, but that is its own endeavor

In [ ]:
encoded_cols_df = pd.get_dummies(df[["REGION", "EVENT_TYPE"]]).astype(int)

encoded_cols_df.head()

In [ ]:
preproc_df = pd.merge(df, encoded_cols_df, on=df.index).drop(columns=["REGION", "EVENT_TYPE", "key_0"])

preproc_df.head()

In [ ]:
output_path=#Replace with intended output path
preproc_df.to_csv(output_path)